#Train Caffe model for validation

In [1]:
import os
os.chdir('../../')
import sys
import time
sys.path.insert(0, './python')
sys.path.append('/local-scratch/xca64/tmp/caffe-master/python/myFunc')
import caffe
import numpy as np
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

caffe.set_device(0)
caffe.set_mode_gpu()

In [2]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

import tempfile

In [3]:
def run_solver(solver, niter, disp_interval, test_interval):
    blobs = ('loss', 'acc')
    loss, acc = (np.zeros(niter), np.zeros(niter))
    test_loss = (np.zeros(niter/test_interval)) 
    for it in range(niter):
        solver.step(1)  # run a single SGD step in Caffe
        loss[it] = (solver.net.blobs['loss'].data.copy())
        acc[it] = 0#(solver.net.blobs['loss_ang'].data.copy())
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = 'loss: %.3f'%loss[it]
            print '%3d) %s Angular Erro %.3f' % (it, loss_disp, acc[it])     
            #print(solver.net.blobs['fc8_flickr'].data[1], solver.net.blobs['illu'].data[1])
        # if it % test_interval == 0:
        #     print "%3d) test loss is %.3f"% (it ,solver.test_nets[0].blobs['accuracy'].data.copy())
        #     test_loss[it//test_interval] = solver.test_nets[0].blobs['accuracy'].data.copy()
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    name = 'firstTry'
    weights = {}
    filename = 'weights.%s.caffemodel' % name
    
    weights[name] = os.path.join(weight_dir, filename)
    solver.net.save(weights[name])
    
    return loss, test_loss, acc, weights

In [5]:
niters = 100
loss_1, test_loss, acc_1, weights_1 = run_solver(solver, niters,20, 5000)

  0) loss: 12.764 Angular Erro 0.000
 20) loss: 5.451 Angular Erro 0.000
 40) loss: 3.802 Angular Erro 0.000
 60) loss: 4.513 Angular Erro 0.000
 80) loss: 6.681 Angular Erro 0.000
 99) loss: 5.572 Angular Erro 0.000


#Save caffe Model for validation

In [5]:
solver.net.save('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')
#save trainning loss for writing up
#np.save('models/color_constancy/result/loss_1.npy', loss_1)

#Load validation data and net

In [6]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver_validation.prototxt')
solver.net.copy_from('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')
from colorConstancy import * 

In [38]:
validationIters = 2 #every time forward a batch size image
validationSetSize = 168 #number of validation images
allAngulareErr = []
for it in range(validationIters):
    solver.step(1)
    estimatedIllu = solver.net.blobs['fc8_flickr'].data.copy()  #batch size illumination estimation result
    groundtruethIllu = solver.net.blobs['illu'].data.copy()  #batch size illumination estimation result
    allAngulareErr.append(list(np.squeeze(np.transpose(multiangle(estimatedIllu, groundtruethIllu)))))  

In [39]:
print np.shape(allAngulareErr)

(2, 50)


#Compute the 10th, median, average, 75th, 90th percitile